<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/text-generation-webui/Colab_LoRA_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM text generation notebook for Google Colab

This notebook uses https://github.com/utensil/text-generation-webui/tree/dev to train LoRa.

Run all the cells and a public gradio URL will appear at the bottom in around 5 minutes.

## Parameters

* **save_logs_to_google_drive**: saves your chat logs, characters, and softprompts to Google Drive automatically, so that they will persist across sessions.
* **text_streaming**: streams the text output in real time instead of waiting for the full response to be completed.
* **cai_chat**: makes the interface look like Character.AI. Otherwise, it looks like a standard WhatsApp-like chat.
* **load_in_8bit**: loads the model with 8-bit precision, reducing the GPU memory usage by half. This allows you to use the full 2048 prompt length without running out of memory, at a small accuracy and speed cost.
* **activate_silero_text_to_speech**: responses will be audios instead of text. There are 118 voices available (`en_0` to `en_117`), which can be set in the "Extensions" tab of the interface. You can find samples here: [Silero samples](https://oobabooga.github.io/silero-samples/).
* **activate_sending_pictures**: adds a menu for sending pictures to the bot, which are automatically captioned using BLIP.
* **activate_character_bias**: an extension that adds an user-defined, hidden string at the beginning of the bot's reply with the goal of biasing the rest of the response.
* **chat_language**: if different than English, activates automatic translation using Google Translate, allowing you to communicate with the bot in a different language.

## Credits

Based on https://colab.research.google.com/github/oobabooga/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb from https://github.com/oobabooga/text-generation-webui/wiki/Running-on-Colab.


In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
%cd /content
!git clone https://github.com/utensil/text-generation-webui
!git checkout dev
!mkdir text-generation-webui/logs

%cd text-generation-webui
!pip install -r requirements.txt

/content
fatal: destination path 'text-generation-webui' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
mkdir: cannot create directory ‘text-generation-webui/logs’: File exists
/content/text-generation-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jtas1923
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jtas1923
  Resolved https://github.com/huggingface/transformers to commit cd73b9a8c140fb74cd93187f5c3d380cfc308023
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!ls

api-example.py		   extensions  prompts
api-example-stream.py	   LICENSE     README.md
characters		   logs        requirements.txt
convert-to-flexgen.py	   loras       server.py
convert-to-safetensors.py  models      settings-template.json
css			   modules     softprompts
download-model.py	   presets     training


In [ ]:
!python download-model.py decapoda-research/llama-7b-hf

100% 8.31k/8.31k [00:00<00:00, 4.68MiB/s]
100% 427/427 [00:00<00:00, 422kiB/s]
100% 124/124 [00:00<00:00, 120kiB/s]
100% 405M/405M [00:04<00:00, 93.7MiB/s]
100% 405M/405M [00:05<00:00, 69.4MiB/s]
100% 405M/405M [00:04<00:00, 86.2MiB/s]
100% 405M/405M [00:04<00:00, 95.0MiB/s]
100% 405M/405M [00:06<00:00, 65.4MiB/s]
100% 405M/405M [00:04<00:00, 95.9MiB/s]
100% 405M/405M [00:04<00:00, 94.3MiB/s]
100% 405M/405M [00:05<00:00, 77.3MiB/s]
100% 405M/405M [00:04<00:00, 95.5MiB/s]
100% 405M/405M [00:06<00:00, 63.3MiB/s]
100% 405M/405M [00:04<00:00, 95.3MiB/s]
100% 405M/405M [00:04<00:00, 88.1MiB/s]
100% 405M/405M [00:04<00:00, 82.2MiB/s]
100% 405M/405M [00:04<00:00, 96.3MiB/s]
100% 405M/405M [00:05<00:00, 68.1MiB/s]
100% 405M/405M [00:04<00:00, 96.8MiB/s]
100% 405M/405M [00:04<00:00, 83.8MiB/s]
100% 405M/405M [00:04<00:00, 85.5MiB/s]
100% 405M/405M [00:04<00:00, 94.8MiB/s]
100% 405M/405M [00:06<00:00, 66.4MiB/s]
100% 405M/405M [00:04<00:00, 94.6MiB/s]
100% 405M/405M [00:05<00:00, 77.4MiB/s]
100%

In [ ]:
!ls -1 models/**/tokenizer_config.json|xargs sed -i -e 's/LLaMATokenizer/LlamaTokenizer/g'

In [ ]:
%cd training/datasets/
!wget https://raw.githubusercontent.com/tloen/alpaca-lora/main/alpaca_data_cleaned.json

/content/text-generation-webui/training/datasets
--2023-03-30 08:02:51--  https://raw.githubusercontent.com/tloen/alpaca-lora/main/alpaca_data_cleaned.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22680910 (22M) [text/plain]
Saving to: ‘alpaca_data_cleaned.json.2’

alpaca_data_cleaned 100%[===================>]  21.63M  --.-KB/s    in 0.1s    

2023-03-30 08:02:53 (217 MB/s) - ‘alpaca_data_cleaned.json.2’ saved [22680910/22680910]



In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/3b.txt /content/text-generation-webui/training/datasets/

In [ ]:
!ls /content/text-generation-webui/training/datasets

In [ ]:
!head /content/text-generation-webui/training/datasets/3b.txt -n 300 | tee /content/text-generation-webui/training/datasets/3b-300.txt

In [ ]:
!python download-model.py tloen/alpaca-lora-7b

100% 791/791 [00:00<00:00, 571kiB/s]
100% 399/399 [00:00<00:00, 344kiB/s]
100% 67.2M/67.2M [00:01<00:00, 64.6MiB/s]


In [ ]:
!python download-model.py  silk-road/luotuo-lora-7b-1.0

100% 28.0/28.0 [00:00<00:00, 20.2kiB/s]
100% 370/370 [00:00<00:00, 256kiB/s]
100% 16.8M/16.8M [00:02<00:00, 6.31MiB/s]


In [ ]:
%cd /content/text-generation-webui
!python server.py --model llama-7b-hf --lora luotuo-lora-7b-1.0 --cai-chat --load-in-8bit --share

/content/text-generation-webui

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//c

In [ ]:
%cd /content/text-generation-webui
!python server.py --model llama-7b-hf --lora test --cai-chat --load-in-8bit --share

/content/text-generation-webui

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//c

In [ ]:
!ls loras

place-your-loras-here.txt  test
